In [4]:
# pip install mysql-connector

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 42.4 MB/s eta 0:00:0000:010:01
  Preparing metadata (setup.py) ... done
  Created wheel for mysql-connector: filename=mysql_connector-2.2.9-cp37-cp37m-linux_x86_64.whl size=247948 sha256=529b00494d415075dfebaf81795d3dc5a798c93f9eeae0d09aca65f2b631d12b
  Stored in directory: /root/.cache/pip/wheels/42/2f/c3/692fc7fc1f0d8c06b9175d94f0fc30f4f92348f5df5af1b8b7
Successfully built mysql-connector
DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
import pandas as pd
import mysql.connector

In [3]:
# Database connection parameters
db_config = {
    'host': 'proxx-rtls-cv-prototype.cxg1ksdtmfbm.eu-west-2.rds.amazonaws.com',
    'user': 'riv',
    'password': 'Call me Ishmae1,',
    'database': 'disease_transmission_simulation',
    'port': 3306
}

In [4]:
# Connect to the database
conn = mysql.connector.connect(**db_config)

# Check if the connection is open
if conn.is_connected():
    print("Connection is open.")
else:
    print("Connection is closed.")

Connection is open.


In [5]:
N = 468  # Total number of individuals

# Create the dictionary to store person data
people_dict = {}
for person_id in range(1, N+1):
    people_dict[person_id] = {
        'infection_status': 0,
        'incubation_period': 0
    }

# Print the first few entries of the dictionary
for person_id in range(1, 11):  # Print the first 10 entries
    print(f"Person ID: {person_id}, Data: {people_dict[person_id]}")

Person ID: 1, Data: {'infection_status': 0, 'incubation_period': 0}
Person ID: 2, Data: {'infection_status': 0, 'incubation_period': 0}
Person ID: 3, Data: {'infection_status': 0, 'incubation_period': 0}
Person ID: 4, Data: {'infection_status': 0, 'incubation_period': 0}
Person ID: 5, Data: {'infection_status': 0, 'incubation_period': 0}
Person ID: 6, Data: {'infection_status': 0, 'incubation_period': 0}
Person ID: 7, Data: {'infection_status': 0, 'incubation_period': 0}
Person ID: 8, Data: {'infection_status': 0, 'incubation_period': 0}
Person ID: 9, Data: {'infection_status': 0, 'incubation_period': 0}
Person ID: 10, Data: {'infection_status': 0, 'incubation_period': 0}


In [6]:
seed_value = 112  # You can choose any seed value
np.random.seed(seed_value)

# Create initial infection status (0: susceptible, 1: infected)
initial_infected_id = np.random.choice(N, size=5, replace=False)
print("First person that has been infected is person id ", initial_infected_id)

First person that has been infected is person id  [109 441 419 137 196]


In [7]:
seed_value = 19
np.random.seed(seed_value)

mean = 1440
std_dev = 576
# mean = 20
# std_dev = 2

# Number of random numbers to generate
num_random_numbers = 5

# Generate an array of random integers with the specified mean and standard deviation
recovery_times = np.random.randint(mean - std_dev, mean + std_dev + 1, num_random_numbers)
print(recovery_times)


[1486 1621 1800 1907 1172]


In [8]:
# Initial infection
# Update infection status for each ID
for person_id in initial_infected_id:
    if person_id in people_dict:
        people_dict[person_id]['infection_status'] = 1
        

for person_id, recovery_time in zip(initial_infected_id, recovery_times):
    if person_id in people_dict:
        people_dict[person_id]['incubation_period'] = recovery_time
        
# print([person_id for person_id, data in people_dict.items() if data['infection_status'] == 1])
people_with_infection = [(person_id, data) for person_id, data in people_dict.items() if data['infection_status'] == 1]
print(people_with_infection)



[(109, {'infection_status': 1, 'incubation_period': 1486}), (137, {'infection_status': 1, 'incubation_period': 1907}), (196, {'infection_status': 1, 'incubation_period': 1172}), (419, {'infection_status': 1, 'incubation_period': 1800}), (441, {'infection_status': 1, 'incubation_period': 1621})]


In [16]:
# Store infection status over time in a dictionary of dictionaries
infection_status_over_time = {}
infection_status_over_time[0] = {person_id: {'infection_status': data['infection_status'],
                                                  'incubation_period': data['incubation_period']}
                                     for person_id, data in people_dict.items()}

print(infection_status_over_time[0].get(441))

{'infection_status': 1, 'incubation_period': 1621}


In [17]:
beta_0 = 0.00055  # 0.0000405      # exogenous/external transmission rate (optimal 0.1-0.2)
beta_1 = 0.003  # 0.00005     # network transmission rate given contact (optimal 30)
beta_2 = 0.003    #0.00003 # network transmission rate given contact


In [18]:
# Simulate disease spread and recovery over time steps
num_time_steps = 4032
should_break = False  # Flag to determine if the loop should be exited

for t in range(1, num_time_steps+1):
    
    # Finds labels from past timestep
    prev_timestep_data = infection_status_over_time[t - 1]
    
    # Identifies individuals that are susceptible and infected from past timestep
    susceptible = [person_id for person_id, data in prev_timestep_data.items() if data['infection_status'] == 0]
    infected = [person_id for person_id, data in prev_timestep_data.items() if data['infection_status'] == 1]
    
    
    # Handles case when no more infected people in population
    # Assumes infection status remains the same for all remaining timesteps
    if not infected and not susceptible:
        print("No infected or susceptible people in population from time", t, "onwards")
        recovered = [person_id for person_id, data in prev_timestep_data.items() if data['infection_status'] == 2]
        print("Number of recovered people in the population is: ", len(recovered))
        
        should_break = True  # Set the flag to break the loop
        
        for time in range(t, num_time_steps + 1):
              infection_status_over_time[time] = {person_id: {'infection_status': data['infection_status'], 'incubation_period': data['incubation_period']} 
                                                  for person_id, data in prev_timestep_data.items()}
    
    if should_break:
        break  # Check the flag and break the loop if necessary
        
    new_infected = []
    
    query = f"""
    SELECT *
    FROM distinct_records_sorted
    WHERE timestep = {t}
    """

    # Fetch data into a Pandas DataFrame
    data_t = pd.read_sql(query, conn)
    
    # Extract unique numbers from min_person_id and max_person_id columns
    # num_people = len(pd.concat([data_t['min_person_id'], data_t['max_person_id']]).unique())
    num_people = N

    # print(num_people)
    
    # Calculate FoI based on interactions
    current_FoI = np.zeros(len(susceptible))
    for idx, s in enumerate(susceptible):
        # print("Person id: ", idx)
        interactions = data_t[((data_t['min_person_id'] == s) | (data_t['max_person_id'] == s)) &
                          (data_t['min_person_id'].isin(infected) | data_t['max_person_id'].isin(infected))]
        
        if len(interactions) > 0:
            distances = interactions['distance'].values
            if len(distances) == 1:
                # print("one interaction")
                current_FoI[idx] = beta_0 + beta_1 / distances[0] + beta_2*(len(infected)/num_people)
            else:
                # print("multiple interactions", len(interactions))
                # FoI_sum = np.sum(beta_0 + beta_1 / distances)
                current_FoI[idx] = beta_0 + beta_1*(np.sum(1 / distances)) + beta_2*(len(infected)/num_people)
            
            # print("Distance: ", distances)
            # print("FoI: ", FoI[idx])

        else:
            current_FoI[idx] = beta_0 + beta_2*(len(infected)/num_people)
            
            # print("Distance: ", 0)
            # print("FoI: ", FoI[idx])

    # Calculate probability of infection
    # Calculate p_infect as y = 1 - e^(-0.5x)
    p_infect = 1 - np.exp(-0.2 * current_FoI)
    
    # Generate random numbers for comparison
    # seed_value = 111
    # np.random.seed(seed_value)
    rnd = np.random.uniform(0, 1, len(susceptible))

    # Determine new infected individuals
    # if len(infected) <= 250:
    new_infected_indices = np.where(rnd <= p_infect)[0]
    new_infected = [susceptible[idx] for idx in new_infected_indices]

    recovery_times = np.random.normal(mean, std_dev, len(new_infected))
    recovery_times = np.clip(recovery_times, 0, None)  # Clip negative values to zero
    recovery_times = np.around(recovery_times)
    # else:
#         new_infected = []
#         recovery_times = []
        
    
    # print("Number of new infections ", {t}, len(new_infected))
    if t % 20 == 0:
        print("Number of infected people present at the beginning of the current timestep ", {t},  len(infected))
        # print(np.max(p_infect))
        print(len(new_infected))

    # Replicate infection status for current timestep
    infection_status_over_time[t] = {
        person_id: {
            'infection_status': infection_status_over_time[t - 1][person_id]['infection_status'],
            'incubation_period': infection_status_over_time[t - 1][person_id]['incubation_period']
        }
        for person_id in range(1, N + 1)
    }
    
    # Update infection status and recovery time for new infected individuals
    for person_id, recovery_time in zip(new_infected, recovery_times):
        infection_status_over_time[t][person_id]['infection_status'] = 1
        infection_status_over_time[t][person_id]['incubation_period'] = recovery_time

    
    # Decrease incubation period for individuals with positive incubation periods
    new_recoveries = 0
    for person_id, data in infection_status_over_time[t].items():
        if data['incubation_period'] > 0:
            data['incubation_period'] -= 1
            if data['incubation_period'] == 0 and data['infection_status'] == 1:
                infection_status_over_time[t][person_id]['infection_status'] = 2
                new_recoveries += 1

/tmp/ipykernel_5409/2059205782.py:40: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_t = pd.read_sql(query, conn)


Number of infected people present at the beginning of the current timestep  {20} 6
0
Number of infected people present at the beginning of the current timestep  {40} 7
0
Number of infected people present at the beginning of the current timestep  {60} 9
0
Number of infected people present at the beginning of the current timestep  {80} 11
0
Number of infected people present at the beginning of the current timestep  {100} 13
0
Number of infected people present at the beginning of the current timestep  {120} 16
0
Number of infected people present at the beginning of the current timestep  {140} 18
0
Number of infected people present at the beginning of the current timestep  {160} 18
0
Number of infected people present at the beginning of the current timestep  {180} 19
0
Number of infected people present at the beginning of the current timestep  {200} 21
0
Number of infected people present at the beginning of the current timestep  {220} 21
0
Number of infected people present at the beginning

In [19]:
# Find individuals who did not have infection_status of 2 at the final timestep
individuals_without_status_2 = []

final_timestep = max(infection_status_over_time.keys())
print(final_timestep)

for person_id, data in infection_status_over_time[final_timestep].items():
    if data['infection_status'] == 0:
        individuals_without_status_2.append(person_id)

print("Individuals without infection status 2 at the final timestep:", len(individuals_without_status_2))

4032
Individuals without infection status 2 at the final timestep: 105


In [20]:
# Create a dictionary to store infection status lists for each time step
infection_status_lists = {}

# Iterate through the dictionary and extract infection status lists
for t, data_dict in infection_status_over_time.items():
    infection_status_lists[t] = [data['infection_status'] for person_id, data in data_dict.items()]

# Convert the dictionary into a pandas DataFrame
df = pd.DataFrame(infection_status_lists)

# Replace all occurrences of 2 with 0
df = df.replace(2, 0)

print(df)

     0     1     2     3     4     5     6     7     8     9     ...  4023   
0       0     0     0     0     0     0     0     0     0     0  ...     0  \
1       0     0     0     0     0     0     0     0     0     0  ...     0   
2       0     0     0     0     0     0     0     0     0     0  ...     0   
3       0     0     0     0     0     0     0     0     0     0  ...     0   
4       0     0     0     0     0     0     0     0     0     0  ...     0   
..    ...   ...   ...   ...   ...   ...   ...   ...   ...   ...  ...   ...   
463     0     0     0     0     0     0     0     0     0     0  ...     0   
464     0     0     0     0     0     0     0     0     0     0  ...     0   
465     0     0     0     0     0     0     0     0     0     0  ...     1   
466     0     0     0     0     0     0     0     0     0     0  ...     0   
467     0     0     0     0     0     0     0     0     0     0  ...     0   

     4024  4025  4026  4027  4028  4029  4030  4031  4032  
0  

In [21]:
# Extract the last row
last_row = df.iloc[-1:]

# Remove the last row from the original DataFrame
df = df.iloc[:-1]

# Insert the last row as the first row
df = pd.concat([last_row, df], ignore_index=True)

In [22]:
print(df)

     0     1     2     3     4     5     6     7     8     9     ...  4023   
0       0     0     0     0     0     0     0     0     0     0  ...     0  \
1       0     0     0     0     0     0     0     0     0     0  ...     0   
2       0     0     0     0     0     0     0     0     0     0  ...     0   
3       0     0     0     0     0     0     0     0     0     0  ...     0   
4       0     0     0     0     0     0     0     0     0     0  ...     0   
..    ...   ...   ...   ...   ...   ...   ...   ...   ...   ...  ...   ...   
463     0     0     0     0     0     0     0     0     0     0  ...     0   
464     0     0     0     0     0     0     0     0     0     0  ...     0   
465     0     0     0     0     0     0     0     0     0     0  ...     0   
466     0     0     0     0     0     0     0     0     0     0  ...     1   
467     0     0     0     0     0     0     0     0     0     0  ...     0   

     4024  4025  4026  4027  4028  4029  4030  4031  4032  
0  

In [23]:
# Save the DataFrame to a CSV file
df.to_csv('virus-slow75.csv', index=False)

In [41]:
%store df

Stored 'df' (DataFrame)


In [ ]:
for t in range(0, 100):  # Loop through the first 10 timesteps
    person_data = infection_status_over_time[t][224]  # Access person ID 396 data for current timestep
    infection_status = person_data['infection_status']
    incubation_period = person_data['incubation_period']
    print(f"Time Step: {t}, Infection Status for Person 396: {infection_status}, Incubation Period: {incubation_period}")

# [9, 78, 90, 104, 128, 131, 146, 266, 318, 322]


In [459]:
susceptible_at_timestep_30 = [person_id for person_id, data in infection_status_over_time[30].items() if data['infection_status'] == 0]
print("Individuals with infection status 0 at timestep 30:", susceptible_at_timestep_30)


Individuals with infection status 0 at timestep 30: [224]


In [ ]:
infection_status_over_time[3]

In [ ]:
# Print the infection status over time (nested list)
# for t, status in enumerate(infection_status_over_time):
#     print(f"Time step {t}:")
#     print(f"{status}")
#     print()

In [42]:
# Close the connection
conn.close()

# Check if the connection is open
if conn.is_connected():
    print("Connection is open.")
else:
    print("Connection is closed.")

Connection is closed.


In [44]:
result_data = []

for t in range(0, 400):
    
    query = f"""
        SELECT *
        FROM distinct_records_sorted
        WHERE timestep = {t}
        """

    # Fetch data into a Pandas DataFrame
    data_t = pd.read_sql(query, conn)

    # Extract unique numbers from min_person_id and max_person_id columns
    people_ids = pd.concat([data_t['min_person_id'], data_t['max_person_id']]).unique()
    people_ids = sorted(people_ids) 
    result_data.append({'timestep': t, 'unique_people_ids': people_ids})
    
# Create a DataFrame from the result_data list
result_df = pd.DataFrame(result_data)
# print(result_df)
# Save the DataFrame to a CSV file
result_df.to_csv('unique_people_ids_per_timestep.csv', index=False)        